In [2]:
import numpy as np
import pickle
import pandas as pd
import torch
from gensim.models import CoherenceModel, LdaModel
from gensim import corpora

In [3]:
with open("discover_train_brand_index.pickle", 'rb') as f:
    train_brand_ids = pickle.load(f)
with open("discover_test_brand_index.pickle", 'rb') as f:
    test_brand_ids = pickle.load(f)
with open("discover_train_inf_index.pickle", 'rb') as f:
    train_inf_ids = pickle.load(f)
with open("discover_test_inf_index.pickle", 'rb') as f:
    test_inf_ids = pickle.load(f)

with open("train_test_split/train_node_brand_id.pkl", 'rb') as f:
    train_node_brand_id = pickle.load(f)
with open("train_test_split/train_node_inf_id.pkl", 'rb') as f:
    train_node_inf_id = pickle.load(f)
with open("train_test_split/test_node_brand_id.pkl", 'rb') as f:
    test_node_brand_id = pickle.load(f)
with open("train_test_split/test_node_inf_id.pkl", 'rb') as f:
    test_node_inf_id = pickle.load(f)

with open("train_test_split/train_brand_node_id.pkl", 'rb') as f:
    train_brand_node_id = pickle.load(f)
with open("train_test_split/train_inf_node_id.pkl", 'rb') as f:
    train_inf_node_id = pickle.load(f)
with open("train_test_split/test_brand_node_id.pkl", 'rb') as f:
    test_brand_node_id = pickle.load(f)
with open("train_test_split/test_inf_node_id.pkl", 'rb') as f:
    test_inf_node_id = pickle.load(f)

with open('sorted_360_brand_list.pkl', 'rb') as f:
    all_brands = pickle.load(f)
with open('sorted_3748_inf_list.pkl', 'rb') as f:
    all_infs = pickle.load(f)
with open('category_list.pickle', 'rb') as f:
    category_list = pickle.load(f)

TRAIN_BRAND_NUM = 286
TRAIN_INF_NUM = 3075
TEST_BRAND_NUM = 74
TEST_INF_NUM = 797

In [6]:
with open("sorted_360_brand_list.pkl", 'rb') as f:
    all_brands = pickle.load(f)
with open("sorted_3748_inf_list.pkl", 'rb') as f:
    all_infs = pickle.load(f)

In [7]:
def get_node_order_id(aid, inf=False):
    if inf:
        node_order_id = test_inf_node_id[aid]-74
    else:
        node_order_id = test_brand_node_id[aid]
    return node_order_id

# Get ranking from predict result

In [ ]:
all_all_scores = torch.load('models/bamirpredict.pt')  

In [ ]:
all_scores = all_all_scores[39] # test result of epoch 40
ranked_influencers = torch.argsort(all_scores, dim=1, descending=True)

In [ ]:
result_aid_dict = dict()
for i in range(74):
    bid = test_node_brand_id[i]
    result_aid_dict[bid] = []
    for j in range(797):
        iid = test_node_inf_id[ranked_influencers[i][j].item()+74]
        result_aid_dict[bid].append(iid)

result_name_dict = dict()
for k, vlist in result_aid_dict.items():
    bname = all_brands[k]
    inames = [all_infs[v] for v in vlist]
    result_name_dict[bname] = inames

In [ ]:
# 載入正合作關係的 tuple (brand, pos_inf)
with open("all_positive_tuples.pickle", 'rb') as f:
    all_pos = pickle.load(f)

In [ ]:
# 篩選出每個品牌對應的正樣本之排名
b_p_rank = dict()
for key, value in result_name_dict.items():
    for i in range(797):
        if (key, value[i]) in all_pos:
            if key not in b_p_rank:
                 b_p_rank[key] = []
            b_p_rank[key].append(i+1)

# Load text, image, node features

In [8]:
train_brand_text_feature = torch.load('train_test_split/train_brand_text_feature_lda.pt')
train_inf_text_feature = torch.load('train_test_split/train_inf_text_feature_lda.pt')
test_brand_text_feature = torch.load('train_test_split/test_brand_text_feature_lda.pt')
test_inf_text_feature = torch.load('train_test_split/test_inf_text_feature_lda.pt')

In [9]:
train_brand_image_feature = torch.load('train_test_split/train_brand_mor_image_feature.pt')
train_inf_image_feature = torch.load('train_test_split/train_inf_mor_image_feature.pt')
test_brand_image_feature = torch.load('train_test_split/test_brand_mor_image_feature.pt')
test_inf_image_feature = torch.load('train_test_split/test_inf_mor_image_feature.pt')

In [10]:
train_brand_node_emb = torch.load('train_test_split/train_brand_hashtag_weighted_nod2vec_0615_r1_sqrt.pt')
test_brand_node_emb = torch.load('train_test_split/test_brand_hashtag_weighted_nod2vec_0615_r1_sqrt.pt')
train_inf_node_emb = torch.load('train_test_split/train_inf_hashtag_weighted_nod2vec_0615_r1_sqrt.pt')
test_inf_node_emb = torch.load('train_test_split/test_inf_hashtag_weighted_nod2vec_0615_r1_sqrt.pt')


# Init LDA

In [11]:
lda_model = LdaModel.load('lda/ldamodel_onlytrain_topic19_it50')

In [13]:
test_corpus = corpora.MmCorpus('lda/onlytrain_test_corpus1.m')
dictionary = corpora.Dictionary.load("lda/onlytrain_dictionary1.dic")

In [15]:
test_corpus_str = []
# Iterate over the corpus
for doc in test_corpus:
    # Convert the sparse vector to text
    text = ' '.join(' '.join([dictionary[id]] * int(freq)) for id, freq in doc)
    
    # Print or process the recovered text
#     print(text)
    test_corpus_str.append(text)
len(test_corpus_str)

871

In [78]:
with open('lda/lda_onlytrain_test_docs_2.pkl', 'rb') as f:
    test_docs_2 = pickle.load(f)

# Init word cloud

In [50]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from PIL import Image

In [51]:
circle_mask = np.array(Image.open("circle_mask.png"))

# Image Label

In [91]:
with open('all_img_label_dict.pkl', 'rb') as f:
    all_img_label_dict = pickle.load(f)

In [85]:
import json
    
# Open the text file
with open('image_vgg/imagenet1000_clsidx_to_labels.txt', 'r') as f:
    labels = [line.strip() for line in f.readlines()]
labels

["{0: 'tench, Tinca tinca',",
 "1: 'goldfish, Carassius auratus',",
 "2: 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',",
 "3: 'tiger shark, Galeocerdo cuvieri',",
 "4: 'hammerhead, hammerhead shark',",
 "5: 'electric ray, crampfish, numbfish, torpedo',",
 "6: 'stingray',",
 "7: 'cock',",
 "8: 'hen',",
 "9: 'ostrich, Struthio camelus',",
 "10: 'brambling, Fringilla montifringilla',",
 "11: 'goldfinch, Carduelis carduelis',",
 "12: 'house finch, linnet, Carpodacus mexicanus',",
 "13: 'junco, snowbird',",
 "14: 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',",
 "15: 'robin, American robin, Turdus migratorius',",
 "16: 'bulbul',",
 "17: 'jay',",
 "18: 'magpie',",
 "19: 'chickadee',",
 "20: 'water ouzel, dipper',",
 "21: 'kite',",
 "22: 'bald eagle, American eagle, Haliaeetus leucocephalus',",
 "23: 'vulture',",
 "24: 'great grey owl, great gray owl, Strix nebulosa',",
 "25: 'European fire salamander, Salamandra salamandra',",
 "26: 'c

In [86]:
resnet_dict = dict()
for i, item in enumerate(labels):
    if i == 0:
        item = item[1:]
    if i == len(labels)-1:
        item = item[:-1]
        
    key, value = item.split(':')
    key = key.strip().strip("'")
    value = value.strip().strip(',').replace("'", "")
    key = int(key)
    resnet_dict[key] = value

In [87]:
resnet_dict

{0: 'tench, Tinca tinca',
 1: 'goldfish, Carassius auratus',
 2: 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 3: 'tiger shark, Galeocerdo cuvieri',
 4: 'hammerhead, hammerhead shark',
 5: 'electric ray, crampfish, numbfish, torpedo',
 6: 'stingray',
 7: 'cock',
 8: 'hen',
 9: 'ostrich, Struthio camelus',
 10: 'brambling, Fringilla montifringilla',
 11: 'goldfinch, Carduelis carduelis',
 12: 'house finch, linnet, Carpodacus mexicanus',
 13: 'junco, snowbird',
 14: 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 15: 'robin, American robin, Turdus migratorius',
 16: 'bulbul',
 17: 'jay',
 18: 'magpie',
 19: 'chickadee',
 20: 'water ouzel, dipper',
 21: 'kite',
 22: 'bald eagle, American eagle, Haliaeetus leucocephalus',
 23: 'vulture',
 24: 'great grey owl, great gray owl, Strix nebulosa',
 25: 'European fire salamander, Salamandra salamandra',
 26: 'common newt, Triturus vulgaris',
 27: 'eft',
 28: 'spotted salamander, Ambystoma 

In [117]:
with open('resnet_dict.pkl', 'wb') as f:
    pickle.dump(resnet_dict, f)

In [88]:
upernet_df = pd.read_csv('image_vgg/unifiedparsing/meta_file/joint_dataset/scene.csv')
upernet_df

,label,name
0,0,airfield
1,1,airplane_cabin
2,2,airport_terminal
3,3,alcove
4,4,alley
...,...,...
360,360,wind_farm
361,361,windmill
362,362,yard
363,363,youth_hostel


In [89]:
all_img_label_dict = resnet_dict.copy()
for i in range(len(upernet_df)):
    all_img_label_dict[1000+i] = upernet_df['name'][i]

In [90]:
all_img_label_dict[1364]

'zen_garden'

In [125]:
with open('all_img_label_dict.pkl', 'wb') as f:
    pickle.dump(all_img_label_dict, f)

# Case Study

## function

In [57]:
with open('brand_pos_inf_ranks.pkl', 'rb') as f:
    b_first_p_rank = pickle.load(f)

In [58]:
wc = WordCloud(background_color="white", max_words=50, collocations=False, mask=circle_mask,
               width=2000, height=1500, colormap='magma') #  plasma  spring
image_wc = WordCloud(background_color="white", max_words=30, collocations=False, mask=circle_mask,
               width=1000, height=750, colormap='magma', min_font_size=20) #  plasma  spring
node_wc = WordCloud(background_color="white", max_words=40, collocations=False, mask=circle_mask,
               width=1000, height=750, colormap='magma', min_font_size=30)

In [64]:
test_brand_names = [all_brands[i] for i in test_brand_ids]
test_inf_names = [all_infs[i] for i in test_inf_ids]

all_test_lda_feat = np.load('lda/lda_topic19_it50_test_pred.npy')

def lda_to_freq(lda_feat):
    term_freq = dict()
    for i in lda_model.show_topics(num_topics=19, formatted=False):
#         print(i[0])
        for (word, weight) in i[1]:
            if word not in term_freq:
                term_freq[word] = 0
            term_freq[word] += weight*lda_feat[i[0]]
    return term_freq

def all_wc_lda(brand_name, inf_names):
    aid = all_brands.index(brand_name)
    iid = test_brand_ids.index(aid)
    lda_aid = test_brand_names.index(brand_name)
    lda_tf = lda_to_freq(all_test_lda_feat[lda_aid])
    wc = WordCloud(background_color="white", max_words=50, collocations=False, mask=circle_mask,
               width=2000, height=1500, colormap='magma', min_font_size=30) #  plasma  spring
    wordcloud = wc.generate_from_frequencies(lda_tf)
    plt.figure(dpi=200)
    plt.imshow(wordcloud)
    plt.axis("off")
    print(f"{brand_name} LDA")
    plt.show()
    
    td = img_to_text_freq(test_brand_image_feature[iid])
    wordcloud = image_wc.generate_from_frequencies(td)
    plt.figure(dpi=200)
    plt.imshow(wordcloud)
    plt.axis("off")
    print(f"{brand_name} image")
    plt.show()

    cd = get_concept_weight(aid, inf=True)
    wordcloud = node_wc.generate_from_frequencies(cd)
    plt.figure(dpi=200)
    plt.imshow(wordcloud)
    plt.axis("off")
    print(f"{brand_name} node")
    plt.show()
    
    
    for inf_name in inf_names:
        aid = all_infs.index(inf_name)
        iid = test_inf_ids.index(aid)+74
        lda_aid = test_inf_names.index(inf_name)+74
        lda_tf = lda_to_freq(all_test_lda_feat[lda_aid])
        wordcloud = wc.generate_from_frequencies(lda_tf)

        plt.figure(dpi=200)
        plt.imshow(wordcloud)
        plt.axis("off")
        print(f"{inf_name} LDA")
        plt.show()
        
        td = img_to_text_freq(test_inf_image_feature[iid])
        wordcloud = image_wc.generate_from_frequencies(td)
        plt.figure(dpi=200)
        plt.imshow(wordcloud)
        plt.axis("off")
        print(f"{inf_name} image")
        plt.show()

        cd = get_concept_weight(aid, inf=True)
        wordcloud = node_wc.generate_from_frequencies(cd)
        plt.figure(dpi=200)
        plt.imshow(wordcloud)
        plt.axis("off")
        print(f"{inf_name} node")
        plt.show()

In [ ]:
all_wc_lda('misspap', ['ajohnson_x', 'stephvillastyle', '_liveymae'])

# Sensitivity Test

## Evaluation Metrics

In [24]:
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score

In [25]:
test_auc_df = pd.read_csv('train_test_split/test_auc.csv')

In [26]:
# Define a function to calculate Recall@k
def recall_at_k(scores, labels, k):
    """
    Calculate Recall@k metric.
    
    Args:
        scores (np.ndarray): 2D array of shape (num_samples, num_classes) containing the predicted scores.
        labels (np.ndarray): 2D array of shape (num_samples, num_classes) containing the binary labels.
        k (int): The value of k for Recall@k.
        
    Returns:
        float: The Recall@k score.
    """
    # Sort the scores in descending order
    sorted_indices = np.argsort(-scores, axis=1)
    
    # Get the top k predicted labels for each sample
    top_k_indices = sorted_indices[:, :k]
    
    # Calculate the number of true positive predictions for each sample
    # true_positives = np.sum(labels[np.arange(labels.shape[0])[:, None], top_k_indices], axis=1)
    true_positives = torch.sum(labels[torch.arange(labels.shape[0])[:, None], top_k_indices], dim=1)
    
    # Calculate the total number of positive labels for each sample
    total_positives = torch.sum(labels, axis=1)
    
    # Calculate Recall@k
    recall_at_k = torch.mean(true_positives / total_positives)
    
    return recall_at_k

In [27]:
def cal_metrics(label_list, scores): # input should be cpu
    # Calculate AUC
    auc = roc_auc_score(label_list.ravel(), scores.ravel())
#     print("AUC:", auc)

    # Calculate Recall@k (e.g. k = 5)
    rec10 = recall_at_k(scores, label_list, 10)
#     print("Recall@10:", rec10.item())
    rec50 = recall_at_k(scores, label_list, 50)
#     print("Recall@50:", rec50.item())

    # Calculate MRR (Mean Reciprocal Rank)
    sorted_indices = np.argsort(-scores, axis=1)
    #   ranks = np.array([np.where(sorted_indices[i] == np.argmax(label_list[i]))[0][0] + 1 for i in range(label_list.shape[0])])
    ranks = np.array([np.argmax(np.isin(sorted_indices[i], np.where(label_list[i] == 1)))+1 for i in range(label_list.shape[0])])
    mrr = np.mean(1 / ranks)
#     print("MRR:", mrr)

    # Calculate MAP (Mean Average Precision)
    map_ = np.mean([average_precision_score(label_list[i], scores[i]) for i in range(label_list.shape[0])])
#     print("MAP:", map_)

    # Calculate MedR (Median Rank)
    medr = np.median(ranks)
#     print("MedR:", medr)
    
    display(pd.DataFrame({'AUC': [auc], 'R@10': [rec10.item()], 'R@50': [rec50.item()], 'MRR': [mrr], 'MAP': [map_], 'MedR': [medr]}))

    return auc, rec10.item(), rec50.item(), mrr, map_, medr


In [28]:
def cal_auc(score, brand_node_id, inf_node_id, auc_df):
    # AUC cAUC
    err = 0
    AUC = 0.0
    AUC_all = 0.0
    cAUC = 0.0
    cAUC_all = 0.0

    brand_num = score.shape[0]

    # iterate through the dataframe
    for i in range(len(auc_df)):
        AUC_all += 1
        score1 = 0.0
        score2 = 0.0
        
        b1_node_id = brand_node_id[auc_df['b1'][i]]  # brand_to_node[brand_id]
        i1_node_id = inf_node_id[auc_df['i1'][i]] - brand_num
        b2_node_id = brand_node_id[auc_df['b2'][i]]
        i2_node_id = inf_node_id[auc_df['i2'][i]] - brand_num
        c1 = auc_df['c1'][i]
        c2 = auc_df['c2'][i]
        
        score1 = score[b1_node_id][i1_node_id]
        score2 = score[b1_node_id][i2_node_id]
        # print(score1, score2)
        
        if (score1 == 0.0 or score2 == 0.0):
            err += 1
        if (c1 == c2):
            cAUC_all += 1
        if (score1 > score2):
            AUC += 1
            if (c1 == c2):
                # print(score1, score2)
                cAUC += 1

    print('AUC:', AUC/AUC_all)
    print('cAUC:', cAUC/cAUC_all)
    print(err)
    print(AUC_all, cAUC_all)
    return AUC / AUC_all, cAUC / cAUC_all

In [367]:
from scipy.stats import rankdata

In [407]:
def cal_metrics_same_score(label_list, scores): # input should be cpu
    # Calculate AUC
    auc = roc_auc_score(label_list.ravel(), scores.ravel())
#     print("AUC:", auc)

    # Calculate Recall@k (e.g. k = 5)
    rec10 = recall_at_k(scores, label_list, 10)
#     print("Recall@10:", rec10.item())
    rec50 = recall_at_k(scores, label_list, 50)
#     print("Recall@50:", rec50.item())

    # Calculate MRR (Mean Reciprocal Rank)
    sorted_indices = np.argsort(-scores, axis=1)
#     print(sorted_indices)
    #   ranks = np.array([np.where(sorted_indices[i] == np.argmax(label_list[i]))[0][0] + 1 for i in range(label_list.shape[0])])
    ranks_org = np.array([np.argmax(np.isin(sorted_indices[i], np.where(label_list[i] == 1)))+1 for i in range(label_list.shape[0])])
#     print(ranks)
    
    sorted_indices = rankdata(-scores, method='min', axis=1)
#     print(sorted_indices.shape, sorted_indices)
    ranks = np.array([np.min(sorted_indices[i][np.where(label_list[i] == 1)]) for i in range(label_list.shape[0])])
    print(np.array_equal(ranks_org, ranks))
    
    mrr = np.mean(1 / ranks)
#     print("MRR:", mrr)

    # Calculate MAP (Mean Average Precision)
    map_ = np.mean([average_precision_score(label_list[i], scores[i]) for i in range(label_list.shape[0])])
#     print("MAP:", map_)

    # Calculate MedR (Median Rank)
    medr = np.median(ranks)
#     print("MedR:", medr)
    
    display(pd.DataFrame({'AUC': [auc], 'R@10': [rec10.item()], 'R@50': [rec50.item()], 'MRR': [mrr], 'MAP': [map_], 'MedR': [medr]}))

    return auc, rec10.item(), rec50.item(), mrr, map_, medr


In [ ]:
# Define a function to calculate Recall@k
def recall_at_k(scores, labels, k):
    """
    Calculate Recall@k metric.
    
    Args:
        scores (np.ndarray): 2D array of shape (num_samples, num_classes) containing the predicted scores.
        labels (np.ndarray): 2D array of shape (num_samples, num_classes) containing the binary labels.
        k (int): The value of k for Recall@k.
        
    Returns:
        float: The Recall@k score.
    """
    # Sort the scores in descending order
    sorted_indices = np.argsort(-scores, axis=1)
    
    # Get the top k predicted labels for each sample
    top_k_indices = sorted_indices[:, :k]
    
    # Calculate the number of true positive predictions for each sample
    # true_positives = np.sum(labels[np.arange(labels.shape[0])[:, None], top_k_indices], axis=1)
    true_positives = torch.sum(labels[torch.arange(labels.shape[0])[:, None], top_k_indices], dim=1)
    
    # Calculate the total number of positive labels for each sample
    total_positives = torch.sum(labels, axis=1)
    
    # Calculate Recall@k
    recall_at_k = torch.mean(true_positives / total_positives)
    
    return recall_at_k

# Performance of different categories

In [7]:
with open("brand_category.pkl", 'rb') as f:
    brand_category = pickle.load(f)

In [10]:
all_all_scores = torch.load('models/bamirpredict.pt')

In [18]:
brand_category_nodeid = dict()
for i in range(74):
    brand_id = test_node_brand_id[i]
    cat = brand_category[brand_id]
    if cat not in brand_category_nodeid:
        brand_category_nodeid[cat] = []
    brand_category_nodeid[cat].append(i)

In [22]:
test_label_list = np.load('train_test_split/test_label_list.npy')
test_fixed_label_list = torch.from_numpy(test_label_list)

In [50]:
auc_list = []
cauc_list = []
rec10_list = []
rec50_list = []
mrr_list = []
map_list = []
medr_list = []

for i in range(12):
    brand_ids = brand_category_nodeid[i]
    _, rec10, rec50, mrr, map_, medr = cal_metrics(test_fixed_label_list[brand_ids], all_all_scores[39][brand_ids].cpu())
    
    rec10_list.append(rec10)
    rec50_list.append(rec50)
    mrr_list.append(mrr)
    map_list.append(map_)
    medr_list.append(medr)
    
    brand_acc_ids = [test_node_brand_id[j] for j in brand_ids]
    
    auc_brand_df = test_auc_df[test_auc_df['b1'].isin(brand_acc_ids)].reset_index()
    auc, cauc = cal_auc(all_all_scores[39].cpu(), test_brand_node_id, test_inf_node_id, auc_brand_df)
    auc_list.append(auc)
    cauc_list.append(cauc)
    

,AUC,R@10,R@50,MRR,MAP,MedR
0,0.784428,0.133048,0.328775,0.576471,0.138814,3.0


AUC: 0.7875217202432667
cAUC: 0.5702758979698074
0
55248.0 3842.0


,AUC,R@10,R@50,MRR,MAP,MedR
0,0.553144,0.015152,0.050337,0.181319,0.031429,61.5


AUC: 0.5601202829189784
cAUC: 0.5354077253218884
0
47222.0 2796.0


,AUC,R@10,R@50,MRR,MAP,MedR
0,0.693107,0.15404,0.276671,0.46623,0.121759,3.0


...

AUC: 0.9477884754596786
cAUC: 0.6904885654885655
0
55256.0 3848.0


,AUC,R@10,R@50,MRR,MAP,MedR
0,0.69022,0.137086,0.279739,0.391111,0.115089,3.0


AUC: 0.7141279215328198
cAUC: 0.5842325095718761
0
67901.0 5746.0


,AUC,R@10,R@50,MRR,MAP,MedR
0,0.897166,0.257486,0.613688,0.616037,0.303567,1.0


AUC: 0.895321674348128
cAUC: 0.669891640866873
0
63356.0 5168.0


In [53]:
cat_performance_df = pd.DataFrame(
    {
        'Category': category_list,
        'AUC': auc_list, 
        'cAUC': cauc_list,
        'R@10': rec10_list, 
        'R@50': rec50_list, 
        'MRR': mrr_list,
        'MAP': map_list,
        'MedR': medr_list
    })
cat_performance_df

,Category,AUC,cAUC,R@10,R@50,MRR,MAP,MedR
0,food,0.787522,0.570276,0.133048,0.328775,0.576471,0.138814,3.0
1,electronics,0.560120,0.535408,0.015152,0.050337,0.181319,0.031429,61.5
2,drink,0.710998,0.616402,0.154040,0.276671,0.466230,0.121759,3.0
3,airline,0.818636,0.580777,0.120635,0.301587,0.270440,0.089142,9.0
4,nonprofit,0.838594,0.726082,0.271474,0.506951,0.659722,0.221995,1.5
5,services,0.834010,0.751369,0.164057,0.490727,0.684028,0.210485,1.0
6,shoes,0.945671,0.689279,0.244048,0.806999,0.681818,0.357648,1.5
7,entertainment,0.864627,0.694245,0.096429,0.417063,0.198810,0.134391,9.5
8,clothing,0.854471,0.742670,0.149917,0.426246,0.520085,0.282732,11.0
9,makeup,0.947788,0.690489,0.154817,0.716531,0.288593,0.224246,7.0


In [94]:
cat_performance_df.to_csv('category_performance.csv', index=False)

In [6]:
zip -r bamir_code.zip 

SyntaxError: invalid syntax (4125666854.py, line 1)